<a href="https://colab.research.google.com/github/cedamusk/AI-N-ML/blob/main/SVM_for_Climate_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [140]:
!pip install earthengine-api geemap scikit-learn

In [141]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-climatechangeforedu')

In [142]:
import geemap
import sklearn

In [143]:
dataset = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filter(
    ee.Filter.date('2018-05-01', '2018-05-03')
)
precipitation = dataset.select('precipitation')
precipitation_vis = {
    'min': 1,
    'max': 17,
    'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000'],
}

m = geemap.Map()
m.set_center(17.93, 7.71, 2)
m.add_layer(precipitation, precipitation_vis, 'Precipitation')
m



Map(center=[7.71, 17.93], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [144]:
# Load the dataset
dataset = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').first()

# Define visualization parameters
visualization = {
    'bands': ['temperature_2m'],
    'min': 250,
    'max': 320,
    'palette': [
        '000080', '0000d9', '4000ff', '8000ff', '0080ff', '00ffff',
        '00ff80', '80ff00', 'daff00', 'ffff00', 'fff500', 'ffda00',
        'ffb000', 'ffa400', 'ff4f00', 'ff2500', 'ff0a00', 'ff00ff',
    ]
}

# Create a map centered at the desired location
Map = geemap.Map(center=[45, 70], zoom=3)

# Add the layer
Map.addLayer(dataset, visualization, 'Air temperature (K) at 2m height')

# Display the map
Map


Map(center=[45, 70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [145]:
dataset=ee.ImageCollection('NOAA/CFSV2/FOR6H')\
          .filterDate('2018-03-01', '2018-03-14')

temperature_above_ground=dataset.select('Temperature_height_above_ground')

vis_params={
    'min':220.0,
    'max': 310.0,
    'paletter':['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

Map=geemap.Map(center=[26.4, -88.6], zoom=1)

Map.addLayer(temperature_above_ground.mean(), vis_params, 'Temperature Above Ground')
Map

Map(center=[26.4, -88.6], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [146]:
# Load the dataset
dataset = ee.ImageCollection('RUB/RUBCLIM/LCZ/global_lcz_map/latest').mosaic()

# Define visualization parameters
visualization = {
    'bands': ['LCZ_Filter'],
    'min': 1,
    'max': 17,
    'palette': [
        '8c0000','d10000','ff0000','bf4d00','ff6600',
        'ff9955','faee05','bcbcbc','ffccaa','555555',
        '006a00','00aa00','648525','b9db79','000000',
        'fbf7ae','6a6aff'
    ]
}

# Create a map centered at the specified location
Map = geemap.Map(center=[51.44, 7.26], zoom=6)

# Add the dataset to the map
Map.addLayer(dataset, visualization, 'LCZ_Filter')

# Display the map
Map


Map(center=[51.44, 7.26], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [147]:
dataset = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019').select(
    'discrete_classification'
)

m = geemap.Map()
m.set_center(-88.6, 26.4, 1)
m.add_layer(dataset, {}, 'Land Cover')
m

Map(center=[26.4, -88.6], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [148]:
dataset=ee.ImageCollection('MODIS/061/MCD18C2').filter(ee.Filter.date('2001-01-01', '2001-02-01'))

gmt_1200_par=dataset.select('GMT_1200_PAR')

import folium
from folium import plugins

map=folium.Map(location=[46.529, 6.746], zoom_start=2)

vis_params={
    'min': -236,
    'max':316,
    'palette':['0f17ff', 'b11406', 'f1ff23']
}

image=gmt_1200_par.mosaic()

map_id=image.getMapId(vis_params)
title_url=map_id['tile_fetcher'].url_format

folium.TileLayer(
    tiles=title_url,
    attr='Google Earth Engine',
    name='Total PAR at GMT 12:00',
    overlay=True,
    control=True
).add_to(map)

map

In [149]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [150]:
def initialize_earth_engine():
  try:
    ee.Initialize(project='ee-climatechangeforedu')
  except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='ee-climatechangeforedu')

In [151]:
def fetch_climate_data():
  datasets={
      'precipitation':ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
          .filter(ee.Filter.date('2018-01-01', '2018-12-31')),
      'temperature':ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR'),
      'land_cover':ee.ImageCollection('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019')
  }
  return datasets

In [152]:
def get_african_climate_zones():
  return{
      'Sahara Desert': ee.Geometry.Rectangle(-20, 15, 40, 35),
      'Sahel':ee.Geometry.Rectangle(-20, 10, 40, 15),
      'Tropical Rainforest':ee.Geometry.Rectangle(10, -10, 40, 10),
      'Mediterranean': ee.Geometry.Rectangle(-10, 30, 40, 37),
      'Subtropical':ee.Geometry.Rectangle(10, -35, 50, -10)
  }

In [153]:
def extract_features(datasets, region_of_interest):
  precipitation=datasets["precipitation"].sum().clip(region_of_interest)
  precipitation_stats=precipitation.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=region_of_interest,
      scale=5000
  )

  temperature=datasets['temperature'].select('temperature_2m')
  temp_mean=temperature.mean().clip(region_of_interest)
  temp_std=temperature.reduce(ee.Reducer.stdDev()).clip(region_of_interest)

  temp_stats=temp_mean.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=region_of_interest,
      scale=5000
  )
  temp_std_stats=temp_std.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=region_of_interest,
      scale=5000
  )

  land_cover=datasets['land_cover'].select('discrete_classification')
  land_cover_clipped=land_cover.map(lambda image: image.clip(region_of_interest))
  land_cover_stats=land_cover.reduceRegions(
      reducer=ee.Reducer.countDistinct(),
      geometry=region_of_interest,
      scale=5000
  )

  feature_dict={
      'annual_precipitation': precipitation_stats.get('precipitation').getInfo(),
      'mean_temperature': temp_stats.get('temperature_2m').getInfo(),
      'temperature_variation': temp_std_stats.get('temperature_2m').getInfo(),
      'land_cover_diveristy': land_cover_stats.get('discrete_classification').getInfo()
  }

  return pd.DataFrame([feature_dict])

In [154]:
def prepare_climate_classification_dataset(num_samples=150):
  initialize_earth_engine()
  datasets=fetch_climate_data()
  climate_zones=get_african_climate_zones()

  X=[]
  y=[]

  for zone, geometry in climate_zones.items():
    for _ in range(num_samples//len(climate_zones)):
      features=extract_features(datasets, geometry)
      X.append(features.values[0])
      y.append(zone)

  return np.array(X), np.array(y)


In [155]:
def train_svm_african_climate_classifier():
  X, y=prepare_climate_classification_dataset()

  X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42)

  scaler=StandardScaler()
  X_train_scaled=scaler.fit_transform(X_train)
  X_test_scaled=scaler.transform(X_test)

  svm_classifier={
      'RBf': SVC(kernel='rbf', random_state=42),
      'Linear': SVC(kernel='linear', random_state=42),
      'Polynomial':SVC(kernel='poly', degree=3, random_state=42)
  }

  results={}

  for name, classifier in svm_classifiers.items():
    classifier.fit(X_train_scaled, y_train)
    y_pred=classifier.predict(X_test_scaled)

    print(f"\n{name} Kernel SVM Classification Report:")
    print(classification_report(y_test, y_pred))

    results[name]={
        'model':classifier,
        'accuracy': classification_report(y_test, y_pred, output_dict=True)['accuracy']

    }

  best_model_name=max(results, key=lambda k: results[k]['accuracy'])
  best_model=results[best_model_name]['model']

  return best_model, scaler, best_model_name

In [156]:
def main():
  print("Starting African Climate Classification with SVM")
  model, scaler, best_kernel=train_svm_african_climate_classifier()

  print(f"\nBest performing Model: {best_kernel} Kernel SVM")

  sample_regions={
      'Sahara': [0, 30, 0.1, 1],
      'Tropical': [2000, 27, 2, 10],
      'Mediterranean':[500, 20, 5, 5]
  }

  print("\nSample region Predictions:")
  for region_name, sample_features in sample_regions.items():
    new_sample=np.array([sample_features])
    new_sample_scaled=scaler.transform(new_sample)
    prediction=model.predict(new_sample_scaled)
    print(f"{region_name}: Predicted Climate Zone-{prediction[0]}")

In [157]:
if __name__=="__main__":
  main()

Starting African Climate Classification with SVM


AttributeError: 'ImageCollection' object has no attribute 'reduceRegions'